# 10.0简介

| 方法                            | 定义                      | 特点           |
| ----------------------------- | ----------------------- | ------------ |
| **特征提取 (Feature Extraction)** | 通过创建新特征来降低特征矩阵维度（第9章内容） | 保留原始信息，创造新特征 |
| **特征选择 (Feature Selection)**  | 保留信息量较高的特征，丢弃信息量较低的特征   | 直接筛选已有特征     |


特征选择的三种方法：
过滤器 (Filter)：根据特征的统计信息选择最优特征
包装器 (Wrapper)：通过不断试错，找出能产生高质量预测值的特征子集
嵌入式 (Embedded)：将特征选择作为机器学习算法训练过程的一部分
本章范围：主要讲解过滤器和包装器方法（嵌入式方法将在后续章节讲解）

# 10.1 数值型特征方差的阈值化

问题描述： 从数值型特征中移除方差较小（即可能包含信息较少）的特征

核心思想： 方差阈值化（Variance Thresholding, VT）假设方差小的特征比大方差的特征重要性低

# 方差计算公式

$$
\text{Var}(x) = \frac{1}{n} \sum_{i=1}^n (x_i - \mu)^2
$$

其中：

- $x$ 是特征向量
- $x_i$ 代表一个单独的特征值
- $\mu$ 是特征的均值
- 算出方差后，方差低于阈值的特征会被丢弃

In [24]:
# 加载库
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

# 加载数据
iris = datasets.load_iris()

# 创建features和target
features = iris.data
target = iris.target

# 创建VarianceThreshold对象
thresholder = VarianceThreshold(threshold=0.5)

# 创建大方差特征矩阵
features_high_variance = thresholder.fit_transform(features)

# 显示大方差特征矩阵
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

查看各特征方差：

In [25]:
# 显示方差
thresholder.fit(features).variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

重要注意事项：

| 注意点       | 说明                                            |
| --------- | --------------------------------------------- |
| **单位问题**  | 方差不是中心化的（单位是特征单位的平方）。如果特征单位不同（如年与美元），VT法无法起作用 |
| **阈值选择**  | 方差阈值需手动选择，可通过`variances_`参数查看各特征方差后确定         |
| **标准化问题** | 如果特征已标准化（均值为0，方差为1），VT将不起筛选作用（因为方差全为1）        |


标准化后的方差：

In [3]:
# 加载库
from sklearn.preprocessing import StandardScaler

# 标准化特征矩阵
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

# 计算每个特征的方差
selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

# 10.2 二值特征的方差阈值化

问题描述： 从二值特征数据（只有两个分类）中移除方差较小的特征

二值特征方差公式（伯努利随机变量）：

$$
\text{Var}(x) = p(1 - p)
$$

其中：

- $p$ 是观察值属于第1个分类的概率

In [6]:
# 加载库
from sklearn.feature_selection import VarianceThreshold

# 用如下信息创建特征矩阵：
# 特征0：80%为分类0
# 特征1：80%为分类1
# 特征2：60%为分类0，40%为分类1
features = [[0,1,0],
            [0,1,1],
            [0,1,0],
            [0,1,1],
            [1,0,0]]

# 创建VarianceThreshold对象并运行
thresholder = VarianceThreshold(.75*(1-.75))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

# 10.3 处理高度相关性的特征

问题描述： 特征矩阵中的某些特征具有较高的相关性（冗余特征）

解决方案： 使用相关矩阵检查高相关性特征，删除其中一个

In [8]:
# 加载库
import pandas as pd
import numpy as np

#创建一个特征矩阵，其中包含两个高度相关的特征
features = np.array([[1,1,1],
                     [2,2,0],
                     [3,3,1],
                     [4,4,0],
                     [5,5,1],
                     [6,6,0],
                     [7,7,1],
                     [8,7,0],
                     [9,7,1]])

# 将特征矩阵转换成DataFrame
dataframe = pd.DataFrame(features)

# 创建相关矩阵
corr_matrix = dataframe.corr().abs()

# 选择相关矩阵的上三角阵
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))

# 找到相关性大于0.95的特征列的索引
to_drop = [column for column in upper.columns if any(upper[column]>0.95)]

# 删除特征
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In [9]:
# 相关矩阵
dataframe.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


In [10]:
# 相关矩阵的上三角阵
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


关键发现： 特征0和特征1的相关系数为0.976 > 0.95，需删除其中一个

# 10.4 删除与分类任务不相关的特征

问题描述： 根据分类目标向量，删除信息量较低的特征

解决方案：

| 特征类型      | 统计方法                   | scikit-learn实现 |
| --------- | ---------------------- | -------------- |
| **分类型特征** | 卡方统计量 ($\chi^2$)       | `chi2`         |
| **数值型特征** | 方差分析F值 (ANOVA F-value) | `f_classif`    |


# 卡方统计量公式

$$
\chi^2 = \sum_{i=1}^n \frac{(O_i - E_i)^2}{E_i}
$$

其中：

- $O_i$ 是第 $i$ 个分类的样本数量（观察值）
- $E_i$ 是假设特征 $i$ 和目标向量无关时，期望的第 $i$ 个分类的样本数量

In [14]:
# 加载库
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif

# 加载数据
iris = load_iris()
features = iris.data
target = iris.target

# 将分类数据转换成整数型数据
features = features.astype(int)

# 选择卡方统计量最大的两个特征
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# 显示结果
print('Original features:', features.shape[1])
print('Selected features:', features_kbest.shape[1])

Original features: 4
Selected features: 2


In [15]:
# 选择F值最大的两个特征
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

# 显示结果
print('Original features:', features.shape[1])
print('Selected features:', features_kbest.shape[1])

Original features: 4
Selected features: 2


In [16]:
# 加载库
from sklearn.feature_selection import SelectPercentile

# 选择F值位于前75%的特征
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# 显示结果
print('Original features:', features.shape[1])
print('Selected features:', features_kbest.shape[1])

Original features: 4
Selected features: 3


| 注意点         | 说明                            |
| ----------- | ----------------------------- |
| **适用范围**    | 只能在两个分类数据之间计算，目标向量和特征都必须是分类数据 |
| **数值型特征处理** | 对于数值型特征，需先转换为分类特征后再使用卡方统计     |
| **非负要求**    | 使用卡方统计方法时，所有值都必须是非负的          |


ANOVA F说明：

用来判断每个分类的特征均值之间的差异有多大
如果特征均值差异大，则该特征与目标向量相关，有助于预测

# 10.5 递归式特征消除

问题描述： 自动选择需要保留的最优特征

核心概念：

RFE (Recursive Feature Elimination)：递归式特征消除
RFECV (Recursive Feature Elimination with Cross-Validation)：带交叉验证的递归式特征消除

RFECV流程：

1.使用所有特征训练模型
2.找出具有最小参数（权重/系数）的特征（代表该特征不重要）
3.将该特征从特征集中移除
4.重复上述步骤，使用交叉验证(CV)评估模型性能
5.如果移除特征后模型性能改进，继续循环；如果性能变差，将该特征重新加入
6.最终确定最优特征集

In [19]:
# 加载库
import warnings
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import linear_model

# 忽略一些烦人但是无害的警告信息
warnings.filterwarnings(action='ignore',module = 'scipy',message='^internal gelsd')

# 生成特征矩阵，目标向量以及系数
features,target = make_regression(n_samples=10000,
                                  n_features=100,
                                  n_informative=2,
                                  random_state=1)

# 创建线性回归对象
ols = linear_model.LinearRegression()

# 递归消除特征
rfecv = RFECV(estimator=ols,scoring='neg_mean_squared_error')
rfecv.fit(features,target)
rfecv.transform(features)

array([[ 0.00850799,  0.7031277 , -0.25651883],
       [-1.07500204,  2.56148527, -0.72602474],
       [ 1.37940721, -1.77039484,  0.51485979],
       ...,
       [-0.80331656, -1.60648007,  0.78006511],
       [ 0.39508844, -1.34564911,  1.7983361 ],
       [-0.55383035,  0.82880112, -1.71411248]])

In [20]:
# 最优特征的数量
rfecv.n_features_

3

In [21]:
# 那些特征是最优特征
rfecv.support_

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [22]:
# 将特征从最好到最差排序
rfecv.ranking_

array([33, 83, 73, 32, 22,  1, 76, 15, 46, 93, 39, 57, 89, 80, 38, 70, 34,
       10, 16, 25, 71, 24, 74, 92, 12, 42, 96, 88, 47, 35, 63, 50, 31, 75,
       23, 60, 91, 53, 58,  1, 85, 95, 29, 13, 81, 84, 11, 20, 62, 45, 41,
       59, 66, 78, 98, 77, 52, 94, 56, 30,  1, 67, 19,  6,  2, 54, 40, 27,
       18, 87, 82, 21, 14, 68, 69, 43,  4,  3,  9, 90, 97, 26, 64, 17,  8,
       37, 36, 51, 49, 44, 79, 72, 55,  5,  7, 28, 61, 48, 65, 86])

RFECV关键参数：

| 参数          | 说明                    |
| ----------- | --------------------- |
| `estimator` | 决定训练模型的类型（如线性回归、SVM等） |
| `step`      | 每次迭代丢弃的特征数量或比例        |
| `scoring`   | 交叉验证时评估模型性能的方法        |



重要说明：

RFE假设特征已经进行过标准化处理
该方法假设模型已经包含所有参数（权重或系数），因此适用于线性模型、SVM等有系数或权重的模型
通过交叉验证自动确定最优特征数量，无需手动指定